# 使用 Milvus 和 DeepSeek 构建 RAG

DeepSeek 帮助开发者使用高性能语言模型构建和扩展 AI 应用。它提供高效的推理、灵活的 API 以及先进的专家混合 (MoE) 架构，用于强大的推理和检索任务。

在本教程中，我们将展示如何使用 Milvus 和 DeepSeek 构建一个检索增强生成 (RAG) 管道。#%% md


## S1 准备环境

In [41]:
!pip install pymilvus -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [59]:
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
# 获取 milvus的token
milvus_token = os.getenv("MILVUS_TOKEN")

print(api_key)
print(milvus_token)

sk-4aa800df99d44303a1d9e04e2b677f88
root:Ggi3teD22025allinai#


## S2 准备数据
使用 Milvus 文档 2.4.x 中的 FAQ 页面作为我们 RAG 中的私有知识库，这是一个简单 RAG 管道的良好数据源。

下载 zip 文件并将文档解压到 `milvus_docs` 文件夹。
```
文档链接： https://github.com/milvus-io/milvus-docs/releases/download/v2.4.6-preview/milvus_docs_2.4.x_en.zip

下载后解压到 E://python_learning/milvus_docs
```

我们从 `milvus_docs/en/faq` 文件夹加载所有 markdown 文件。对于每个文档，我们简单地使用 "# " 来分割文件中的内容，这样可以大致分离出 markdown 文件中每个主要部分的内容。

In [11]:
from glob import glob

text_lines = []

for file_path in glob("E://python_learning/milvus_docs/en/faq/*.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")



72

## S3 准备 LLM 和 Embedding 模型

In [12]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [14]:
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()

**测试embedding :**

  两句话 比较下向量 值   This is a test  、 That is a test

In [15]:
test_embedding_1 = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding_1)
print(embedding_dim)
print(test_embedding_1[:10])

768
[-0.04836059  0.07163021 -0.01130063 -0.03789341 -0.03320651 -0.01318453
 -0.03041721 -0.02269495 -0.02317858 -0.00426026]


In [16]:
test_embedding_2 = embedding_model.encode_queries(["That is a test"])[0]
print(test_embedding_2[:10])

[-0.02752976  0.0608853   0.00388525 -0.00215193 -0.02774976 -0.0118618
 -0.04020916 -0.06023417 -0.03813156  0.0100272 ]


## S4 将数据加载到 Milvus

关于 `MilvusClient` 的参数：

*   将 `uri` 设置为本地文件，例如 `./milvus.db`，是最方便的方法，因为它会自动利用 Milvus Lite 将所有数据存储在此文件中。
*   如果您有大规模数据，可以在 Docker 或 Kubernetes 上设置性能更高的 Milvus 服务器。在此设置中，请使用服务器 URI，例如 `http://localhost:19530`，作为您的 `uri`。
*   如果您想使用 Zilliz Cloud（Milvus 的完全托管云服务），请调整 `uri` 和 `token`，它们对应 Zilliz Cloud 中的 Public Endpoint 和 Api key。


 `metric_type`：怎么算相似。
 `consistency_level`：新数据多久能被读到。

In [27]:
!where python

D:\projects\python_projects\dfa_py\.venv\Scripts\python.exe
C:\Users\HP\AppData\Local\Programs\Python\Python313\python.exe
C:\Users\HP\AppData\Local\Microsoft\WindowsApps\python.exe


In [43]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="http://c-6314c3df7e880376.milvus.aliyuncs.com:19530",
                             token=milvus_token,
                             db_name="rag_demo01")

collection_name = "yogi_rag_collection"

# 检查 collection 是否已存在，如果存在则删除它
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

# 如果我们不指定任何字段信息，Milvus 将自动创建一个默认的 `id` 字段作为主键，以及一个 `vector` 字段来存储向量数据。一个保留的 JSON 字段用于存储非 schema 定义的字段及其值。
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 距离度量类型(定义如何计算向量之间的相似程度):
    # 例如：`IP` (内积) - 值越大通常越相似；`L2` (欧氏距离) - 值越小越相似；`COSINE` (余弦相似度) - 通常转换为距离，值越小越相似。
    # 选择依据：根据你的嵌入模型的特性和期望的相似性定义来选择。
    consistency_level="Strong",  # 一致性级别(定义数据写入后，读取操作能多快看到这些新数据。)
    # 例如：
    #     `Strong` (强一致性): 总是读到最新数据，可能稍慢。
    #     `Bounded` (有界过期): 可能读到几秒内旧数据，性能较好 (默认)。
    #     `Session` (会话一致性): 自己写入的自己能立刻读到。
    #     `Eventually` (最终一致性): 最终会读到新数据，但没时间保证，性能最好。
    #        选择依据：在数据实时性要求和系统性能之间做权衡。https://milvus.io/docs/consistency.md#Consistency-Level。
)

In [44]:
# 插入数据
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 72/72 [00:00<00:00, 52419.70it/s]


{'insert_count': 72, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71], 'cost': 0}

## S5 构建 RAG

In [55]:
# 检索
question = "存在milvus中的数据是多少维的?"

search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

In [47]:
# 输出检索结果
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        " Where does Milvus store data?\n\nMilvus deals with two types of data, inserted data and metadata. \n\nInserted data, including vector data, scalar data, and collection-specific schema, are stored in persistent storage as incremental log. Milvus supports multiple object storage backends, including [MinIO](https://min.io/), [AWS S3](https://aws.amazon.com/s3/?nc1=h_ls), [Google Cloud Storage](https://cloud.google.com/storage?hl=en#object-storage-for-companies-of-all-sizes) (GCS), [Azure Blob Storage](https://azure.microsoft.com/en-us/products/storage/blobs), [Alibaba Cloud OSS](https://www.alibabacloud.com/product/object-storage-service), and [Tencent Cloud Object Storage](https://www.tencentcloud.com/products/cos) (COS).\n\nMetadata are generated within Milvus. Each Milvus module has its own metadata that are stored in etcd.\n\n###",
        0.65726637840271
    ],
    [
        "How does Milvus flush data?\n\nMilvus returns success when inserted data are loaded to the

In [56]:
# 使用LLM获取RAG响应
print('question : ' + question)

context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)
print('检索内容转换为 字符串格式 : ' + context)

SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

print('prompt内容 : ' + USER_PROMPT)

question : 存在milvus中的数据是多少维的?
检索内容转换为 字符串格式 :  Where does Milvus store data?

Milvus deals with two types of data, inserted data and metadata. 

Inserted data, including vector data, scalar data, and collection-specific schema, are stored in persistent storage as incremental log. Milvus supports multiple object storage backends, including [MinIO](https://min.io/), [AWS S3](https://aws.amazon.com/s3/?nc1=h_ls), [Google Cloud Storage](https://cloud.google.com/storage?hl=en#object-storage-for-companies-of-all-sizes) (GCS), [Azure Blob Storage](https://azure.microsoft.com/en-us/products/storage/blobs), [Alibaba Cloud OSS](https://www.alibabacloud.com/product/object-storage-service), and [Tencent Cloud Object Storage](https://www.tencentcloud.com/products/cos) (COS).

Metadata are generated within Milvus. Each Milvus module has its own metadata that are stored in etcd.

###
How does Milvus flush data?

Milvus returns success when inserted data are loaded to the message queue. However, the d

In [57]:
# 使用 DeepSeek 提供的 `deepseek-chat` 模型根据提示生成响应。
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据提供的上下文信息，Milvus支持的向量数据类型包括Binary、Float32、Float16和BFloat16，但上下文中没有明确提到存储的向量具体是多少维的。向量的维度通常取决于用户插入数据时的定义，而不是由Milvus本身固定限制。

<translated>
According to the provided context, Milvus supports vector data types including Binary, Float32, Float16, and BFloat16, but the context does not explicitly mention the specific dimensionality of vectors stored. The dimensionality of vectors typically depends on user definitions when inserting data, rather than being fixed by Milvus itself.
</translated>
